In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

## Installing Pytorch3D

In [ ]:
#| export
import subprocess
import sys

import torch


def install_pytorch3d():
    pyt_version_str = torch.__version__.split("+")[0].replace(".", "")
    version_str = "".join(
        [
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".", ""),
            f"_pyt{pyt_version_str}",
        ]
    )
    subprocess.run(["pip", "install", "fvcore", "iopath"])
    subprocess.run(
        [
            "pip",
            "install",
            "--no-index",
            "--no-cache-dir",
            "pytorch3d",
            "-f",
            f"https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html",
        ]
    )

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()